In [65]:
#importing Libraries
import pandas as pd
import numpy as num
import os

In [66]:
path = r'C:\Users\ACER NITRO\Documents\Achievement 4'

In [67]:
df_ords_prods_merged= pd.read_pickle(os.path.join(path,'Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [68]:
df_ords_prods_merged=df_ords_prods_merged.drop(columns=['busiest_day'])

In [69]:
df_ords_prods_merged.shape

(32404859, 18)

In [70]:
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Average orders
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Most orders
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Least busy,Average orders
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Least busy,Most orders


## Step 2: Find the aggregated mean

In [71]:
df_ords_prods_merged.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Step 3: Analyze the result. How do the results for the entire dataframe differ from those of the subset

###  The minimum mean order is 11.3 in the subset while it is 15.2 in the dataframe, and the maximum mean is 19.5 in the subset, 20.1 in the dataframe (not including department 21, the "missing" category).The department ID with the lhighest number of average order numbers is missing in subset, indicating a large number of unspecified values for the column "department_id." There is less variability in the mean orders when calculated using the entire dataframe as comparat to the smaller subset:

## Step4: create a loyalty flag for existing customers using the transform() and loc() functions.

In [72]:
#aggregating data with transform
df_ords_prods_merged['max_order'] = df_ords_prods_merged.groupby('user_id')['order_number'].transform(num.max)

In [73]:
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Average orders,10
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Most orders,10
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Least busy,Average orders,10
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy days,Least busy,Most orders,10


In [74]:
#Deriving columns with loc
df_ords_prods_merged.loc[df_ords_prods_merged['max_order']>40, 'loyalty_flag'] = 'loyal_customer'

In [75]:
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [76]:
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [77]:
df_ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
loyal_customer      10284093
New customer         6243990
Name: count, dtype: int64

## Step5: check the basic statistics of the product prices for each loyalty category

In [78]:
df_ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
New customer,13.294670
Regular customer,12.495717
loyal_customer,10.386336


## Step 6:  Create a spending flag for each user based on the average price across all their orders

In [79]:
#aggregating data with transform
df_ords_prods_merged['mean_price'] = df_ords_prods_merged.groupby('user_id')['prices'].transform(num.mean)

In [80]:
df_ords_prods_merged.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Least busy days,Regularly busy,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Least busy days,Least busy,Most orders,10,New customer,6.367797
5,3367565,1,6,2,7,19.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
6,550135,1,7,1,9,20.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797
7,3108588,1,8,1,14,14.0,196.0,2.0,1.0,Soda,...,7,9.0,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797
8,2295261,1,9,1,16,0.0,196.0,4.0,1.0,Soda,...,7,9.0,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797
9,2550362,1,10,4,8,30.0,196.0,1.0,1.0,Soda,...,7,9.0,both,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797


In [81]:
#Deriving columns with loc
df_ords_prods_merged.loc[df_ords_prods_merged['mean_price']<10, 'spending_flag'] = 'Low_spender'


In [82]:
df_ords_prods_merged.loc[df_ords_prods_merged['mean_price']>10, 'spending_flag'] = 'High_spender'

In [83]:
df_ords_prods_merged['spending_flag'].value_counts(dropna = False)

spending_flag
Low_spender     31770614
High_spender      633864
NaN                  381
Name: count, dtype: int64

## Step7: Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [84]:
# create "median_days" for aggregation results
df_ords_prods_merged['median_days'] = df_ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(num.median)

In [85]:
df_ords_prods_merged.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Least busy days,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Least busy days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797,Low_spender,20.5
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Least busy days,Least busy,Most orders,10,New customer,6.367797,Low_spender,20.5
5,3367565,1,6,2,7,19.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5
6,550135,1,7,1,9,20.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5
7,3108588,1,8,1,14,14.0,196.0,2.0,1.0,Soda,...,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5
8,2295261,1,9,1,16,0.0,196.0,4.0,1.0,Soda,...,both,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5
9,2550362,1,10,4,8,30.0,196.0,1.0,1.0,Soda,...,both,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797,Low_spender,20.5


In [86]:
# Create flag using loc
df_ords_prods_merged.loc[df_ords_prods_merged['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [87]:
df_ords_prods_merged.loc[(df_ords_prods_merged['median_days'] > 10) & (df_ords_prods_merged['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [88]:
df_ords_prods_merged.loc[df_ords_prods_merged['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [89]:
# Check frequencies of spending flags for accuracy
df_ords_prods_merged['frequency_flag'].value_counts(dropna=False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [90]:

pd.options.display.max_rows = None

In [93]:
df_ords_prods_merged.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy days,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy days,Least busy,Most orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
5,3367565,1,6,2,7,19.0,196.0,1.0,1.0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
6,550135,1,7,1,9,20.0,196.0,1.0,1.0,Soda,...,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
7,3108588,1,8,1,14,14.0,196.0,2.0,1.0,Soda,...,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
8,2295261,1,9,1,16,0.0,196.0,4.0,1.0,Soda,...,Mid-range product,Busiest days,Regularly busy,Most orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer
9,2550362,1,10,4,8,30.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy days,Least busy,Average orders,10,New customer,6.367797,Low_spender,20.5,Non-frequent customer


In [92]:
#checking the null values
df_ords_prods_merged[df_ords_prods_merged['frequency_flag'].isnull()]


,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,Busiest_days,Busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag
13645692,895835,159838,1,0,17,NaN,10749.0,3.0,0.0,Organic Red Bell Pepper,...,Mid-range product,Busiest days,Busiest day,Average orders,1,New customer,7.42,Low_spender,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401.0,6.0,0.0,Goat Cheese Crumbles,...,Mid-range product,Busiest days,Busiest day,Average orders,1,New customer,7.42,Low_spender,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695.0,2.0,0.0,California Veggie Burger,...,Low-range product,Busiest days,Busiest day,Average orders,1,New customer,7.42,Low_spender,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334.0,5.0,0.0,Organic Peeled Garlic,...,Mid-range product,Busiest days,Busiest day,Average orders,1,New customer,7.42,Low_spender,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198.0,1.0,0.0,4X Ultra Concentrated Natural Laundry Detergen...,...,Low-range product,Busiest days,Busiest day,Average orders,1,New customer,7.42,Low_spender,NaN,NaN


## Step9: Export your dataframe as a pickle file

In [94]:
df_ords_prods_merged.to_pickle(os.path.join(path, 'Data','Prepared Data', 'orders_products_aggregated.pkl'))